In [10]:
import cv2
import numpy as np
from matplotlib import pyplot as plt 
import tkinter as tk
from tkinter import filedialog
from tkinter import simpledialog
from tkinter import ttk
from PIL import Image, ImageTk

# Image processing functions
def white_balance_correction(image):
    if len(image.shape) == 2 or image.shape[2] == 1:
        # Grayscale image will be returned
        return image

    # Color channels and calculate mean values, scale them
    blue, green, red = cv2.split(image)

    mean_red = np.mean(red)
    mean_blue = np.mean(blue)
    red = (red * mean_blue / mean_red).astype(np.uint8)
    blue = (blue * mean_red / mean_blue).astype(np.uint8)

    # Final balanced image
    balanced_image = cv2.merge([blue, green, red])

    return balanced_image

def tonal_transformations(image, lower_bound, upper_bound):
    # grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # tonal transformation
    transformed_image = cv2.inRange(gray_image, lower_bound, upper_bound)

    # Apply transformation to the original image
    for channel in range(3):
        image[:, :, channel] = cv2.bitwise_and(image[:, :, channel], transformed_image)

    return image

def histogram_equalization(image):
    if len(image.shape) == 2 or image.shape[2] == 1:
        # for grayscale,applied directly
        equalized_image = cv2.equalizeHist(image)
    else:
        # convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        equalized_image = cv2.equalizeHist(gray_image)

    return cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2BGR)  # Convert back to BGR

def contrast_stretching(image, min_intensity, max_intensity):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.normalize(image, None, min_intensity, max_intensity, cv2.NORM_MINMAX)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    return image

def gamma_correction(image, gamma):
    if len(image.shape) == 2 or image.shape[2] == 1:
        # If the image is grayscale, apply gamma correction directly
        gamma_corrected_image = cv2.LUT(image, np.array([pow(i / 255.0, gamma) * 255 for i in np.arange(0, 256)]).astype(np.uint8))
    else:
        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gamma_corrected_gray = cv2.LUT(gray_image, np.array([pow(i / 255.0, gamma) * 255 for i in np.arange(0, 256)]).astype(np.uint8))
        gamma_corrected_image = cv2.cvtColor(gamma_corrected_gray, cv2.COLOR_GRAY2BGR)

    return gamma_corrected_image

class ImageProcessingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Processing App")

        self.image = None
        self.processed_image = None
        self.operations_menu = None 
        self.create_widgets()

    def create_widgets(self):
        # load
        load_button = ttk.Button(self.root, text="Load Image", command=self.load_image)
        load_button.pack()

        # display
        self.image_label = ttk.Label(self.root)
        self.image_label.pack()
        
        # Denoised Image Display
        self.denoised_label = ttk.Label(self.root)
        self.denoised_label.pack()

        # Operations Menu
        operations_label = ttk.Label(self.root, text="Choose an operation:")
        operations_label.pack()

        self.operations_menu = ttk.Combobox(self.root, values=[
            "Contrast Adjustment",
            "White Balance Correction",
            "Tonal Transformation",
            "Histogram Equalization",
            "Contrast Stretching",
            "Gamma Correction"
        ])
        self.operations_menu.set("Choose Option")
        self.operations_menu.pack()

        # Parameters Entry
        self.parameters_entry = None

        # Process Image Button
        process_button = ttk.Button(self.root, text="Process Image", command=self.process_image)
        process_button.pack()

        # Save Image Button
        save_button = ttk.Button(self.root, text="Save Processed Image", command=self.save_image)
        save_button.pack()

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.image = cv2.imread(file_path)
            #self.image = self.resize_image(self.image, target_width=640, target_height=360)
            self.update_image_label()
            
    def resize_image(self, image, target_width, target_height):
        if image is not None:
            image = cv2.resize(image, (target_width, target_height))
        return image

    def update_image_label(self):
        if self.image is not None:
            image_rgb = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
            image_pil = Image.fromarray(image_rgb)
            image_tk = ImageTk.PhotoImage(image_pil)
            self.image_label.config(image=image_tk)
            self.image_label.image = image_tk

    def process_image(self):
        operation = self.operations_menu.get()
            
        if operation == 'Contrast Adjustment':
            alpha = float(simpledialog.askstring("Input", "Enter contrast factor (e.g., 1.5 for increasing contrast):"))
            self.processed_image = cv2.convertScaleAbs(self.image, alpha=alpha, beta=0)
        
        elif operation == 'White Balance Correction':
            self.processed_image = white_balance_correction(self.image)
        
        elif operation == 'Tonal Transformation':
            lower_bound = int(simpledialog.askstring("Input", "Enter lower bound (0-255): "))
            upper_bound = int(simpledialog.askstring("Input", "Enter upper bound (0-255): "))
            self.processed_image = tonal_transformations(self.image, lower_bound, upper_bound)
        
        elif operation == 'Histogram Equalization':
            self.processed_image = histogram_equalization(self.image)
        
        elif operation == 'Contrast Stretching':
            min_intensity = int(simpledialog.askstring("Input", "Enter min intensity (0-255): "))
            max_intensity = int(simpledialog.askstring("Input", "Enter max intensity (0-255): "))
            self.processed_image = contrast_stretching(self.image, min_intensity, max_intensity)
        
        elif operation == 'Gamma Correction':
            gamma = float(simpledialog.askstring("Input", "Enter gamma value: "))
            self.processed_image = gamma_correction(self.image, gamma)
        
        elif operation == 'Save and Exit':
            # Save and exit
            output_path = filedialog.asksaveasfilename(defaultextension=".jpg")
            if output_path:
                cv2.imwrite(output_path, self.processed_image)

        if self.processed_image is not None:
            self.update_image_label()

    def save_image(self):
        if self.processed_image is not None:
            output_path = filedialog.asksaveasfilename(defaultextension=".jpg")
            self.processed_image = cv.fastNlMeansDenoisingColored(self.processed_image,None,10,10,7,21)
            if output_path:
                if self.operations_menu.get() == 'Grayscale Conversion':
                    # If the operation was 'Grayscale Conversion', convert it back to BGR
                    bgr_image = cv2.cvtColor(self.processed_image, cv2.COLOR_GRAY2BGR)
                    cv2.imwrite(output_path, bgr_image)
                else:
                    # For other operations, resize and save the processed image
                    #resized_image = cv2.resize(self.processed_image, (640, 360))
                    cv2.imwrite(output_path, self.processed_image)
                    
if __name__ == "__main__":
    root = tk.Tk()
    app = ImageProcessingApp(root)
    root.mainloop()
